In [1]:
import logging
import sys
import os

import numpy as np

sys.path.insert(0, os.path.abspath('./..'))


import jupylet.color

from jupylet.app import App
from jupylet.state import State
from jupylet.label import Label
from jupylet.sprite import Sprite

from jupylet.audio.sample import Sample

app = App()#log_level=logging.INFO)



background = '#3e32a2'  # la couleur de fond du jeu
foreground = '#7c71da'  # la couleur d'avant plan du jeu (comprend les coté du jeu)


a0 = np.ones((32, 32)) * 255                                         # génération d'une image avec 32 par 32 pixels pour la balle
a1 = np.ones((128, 16)) * 255                                       # génération d'une image 128 par 16 pixels pour les palettes
a2 = np.ones((app.height * 9 // 10, app.width * 9 // 10, 3)) * 255  # génération des pixels de l'image du fond d'écran, le fond d'écran correspond à 9/10 du jeu, donc le 1/10 de chaque côté sera la bordure 


ball = Sprite(a0, y=app.height/2, x=app.width/2)    # image de la balle

padl = Sprite(a1, y=app.height/2, x=48)             # image de la palette de gauche
padr = Sprite(a1, y=app.height/2, x=app.width-48)   # image pour la palette de droite 

field = Sprite(a2, y=app.height/2, x=app.width/2, color=background)  # image pour l'image de font, l'image mesure 90% du jeu (voir variable a2) et est centré dans l'application, on ajoute la couleur de background pour le sprite 


pong_sound = Sample('sounds/pong-blip.wav', amp=0.1).load()  # chargement du son



scorel = Label(
    '0', font_size=42, color=foreground,       # etiquette du résultat à gauche
    x=64, y=app.height/2, 
    anchor_y='center', anchor_x='left',
    font_path='fonts/PetMe64.ttf'
)

scorer = Label(
    '0', font_size=42, color=foreground,       #  etiquette de résultat à droite 
    x=app.width-64, y=app.height/2, 
    anchor_y='center', anchor_x='right',
    font_path='fonts/PetMe64.ttf'
)




@app.event
def render(ct, dt):
    
    app.window.clear(color=foreground)  # on efface tout
    
    field.draw()                        # on dessine le fond du jeu
     
    scorel.draw()                       # on dessine le résultat à gauche
    scorer.draw()                       # on dessine le résultat de droite
    
    ball.draw()                         # on dessine la balle
    padl.draw()                         # on dessine la palette à gauche
    padr.draw()                         # on dessine la palette de droite




state = State(
    
    sl = 0,                             # résultat de gauche
    sr = 0,                             # résultat de droite
    
    bvx = 192,                          # vitesse de la balle horizontale
    bvy = 192,                          # vitesse de la balle verticale
    
    vyl = 0,                            # vitesse de la palette de gauche en vertical
    pyl = app.height/2,                 # position de la palette de gauche 

    vyr = 0,                            # vitesse de la palette de droite en vertical
    pyr = app.height/2,                 # position de la palette de droite en vertical

    left = False,                       # flèche gauche appuyée on non
    right = False,                      # flèche droite appuyée on non

    key_a = False,                      # touche A appuyée ou non
    key_d = False,                      # touche D appuyée ou non
)


@app.event
def key_event(key, action, modifiers):
        
    keys = app.window.keys
    
    if action == keys.ACTION_PRESS:     # événement d'une touche appuyée
        
        if key == keys.LEFT:            # flèche gauche
            state.left = True

        if key == keys.RIGHT:           # flèche droite
            state.right = True

        if key == keys.A:               # touche A
            state.key_a = True

        if key == keys.D:               # touche D
            state.key_d = True

    if action == keys.ACTION_RELEASE:   # événement d'une touche libérée

    
        if key == keys.LEFT:            
            state.left = False

        if key == keys.RIGHT:
            state.right = False

        if key == keys.A:
            state.key_a = False

        if key == keys.D:
            state.key_d = False




@app.run_me_every(1/60)
def update_pads(ct, dt):
        
    if state.right:
        state.pyr = min(app.height, state.pyr + dt * 512)   # assignation de la position de la palette de droite lorsque le déplacement est vers la droite (haut)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)            # assignation de la position de la palette de droite lorsque le déplacement est vers la gauche (bas)
        
    if state.key_a:
        state.pyl = min(app.height, state.pyl + dt * 512)   # assignation de la position de la palette de gauche lorsque le déplacement est vers la gauche (bas)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)            # assignation de la position de la palette de gauche lorsque le déplacement est vers la droite (haut)
        
    ayl = 10 * (state.pyl - padl.y)                         # calcul de l'accélération de la palette gauche
    ayr = 10 * (state.pyr - padr.y)                         # calcul de l'accélération de la palette droite

    
    state.vyl = state.vyl * 0.9 + (ayl * dt)                # calcul de la vitesse de la palette gauche
    state.vyr = state.vyr * 0.9 + (ayr * dt)                # calcul de la vitesse de la palette droite
    
    
    padl.y += state.vyl * dt                                # calcul de la position de la palette gauche
    padr.y += state.vyr * dt                                # calcul de la position de la palette droite
    
    padr.clip_position(app.width, app.height)               # Conserver l'image de la palette droite dans le cadre du jeu
    padl.clip_position(app.width, app.height)               # Conserver l'image de la palette gauche dans le cadre du jeu





@app.run_me_every(1/60)
def update_ball(ct, dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2    # premier calcul de la vitesse horizontale et verticale cumulée
    
    ball.angle += 200 * dt                   # changement de l'angle de la balle (carrée) en proportion de 1/60 seconde 
    
    ball.x += state.bvx * dt                 # déplacement horizontal de la balle sur 1/60 seconde
    ball.y += state.bvy * dt                 # déplacement vertical de la balle sur 1/60 seconde   
    
    if ball.top >= app.height:                                              # vérification si la balle est trop haute 
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note  
        ball.y -= ball.top - app.height                                     # on reposition la balle dans la limite du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement
        
    if ball.bottom <= 0:                                                    # vérification si la balle est trop basse
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note     
        ball.y -= ball.bottom                                               # on ramette la balle dans les limites du jeu
        state.bvy = -state.bvy                                              # on inverse la vitesse pour faire un changement de direction en vertical seulement    
        
    if ball.right >= app.width:                                             # vérification si la balle est trop à droite
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)     # on joue une note
        ball.x -= ball.right - app.width                                    # on repositionne la balle dans le cadre du jeu
        
        state.bvx = -192                                                    # on change la direction horizontale de la vitesse de la balle et on la met à 192
        state.bvy = 192 * np.sign(state.bvy)                                # on assigne la vitesse de la balle à 192 sans changer la direction en verticale
        bs0 = 0                                                             # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sl += 1                                                       # on donne un point au joueur de gauche car la balle a atteint la limite droite
        scorel.text = str(state.sl)                                         # on affiche le score sur le plateau de jeu du joueur de gauche    
        
    if ball.left <= 0:                                                  
        pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)             # On vérifie si la balle est trop à gauche
        ball.x -= ball.left                                                         # on reposition la balle dans la limite du jeu
        
        state.bvx = 192                                                             # on applique la vitesse horizontale de la balle à la valeur par défaut en positif pour que la direction soit vers la droite. 
        state.bvy = 192 * np.sign(state.bvy)                                        # on applique la vitesse verticale de la balle à la valeur par défaut sans changer le signe de la direction verticale
        bs0 = 0                                                                     # on met la vitesse initiale cumulée a 0 lorsque la limite du jeu est atteinte
        
        state.sr += 1                                                               # on donne un point au joueur de droite car la balle a toucher a la limite gauche
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de droite
        if 0 < ball.right - padr.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de droite     
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son
            ball.x -= ball.right - padr.left                                        # on position la balle sur la palette de droite
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyr / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom:       # on vérifie si la balle est dans les limite verticale de la palette de gauche
        if 0 < padl.right - ball.left < 10:                                         # on vérifie si la balle est 10 pixels ou moins près de la palette de gauche
            pong_sound.play(pan=2*max(.25, min(.75, ball.x / app.width))-1)         # on joue un son    
            ball.x += ball.left - padl.right                                        # on position la balle sur la palette de gauche
            state.bvx = -state.bvx                                                  # on inverse la vitesse horizontale de la balle pour faire un changement de direction
            state.bvy += state.vyl / 2                                              # on ajuste la vitesse verticale de la balle en lui ajoutant 50% de la vitesse de la palette droite
            
    bs1 = state.bvx ** 2 + state.bvy ** 2                                           # deuxième calcul de la vitesse horizontale et verticale cumulée
    
    if bs1 < 0.9 * bs0:                                                             # Si le deuxième calcul est plus petit que 90% de la vitesse cumulée, on recalcul la vitesse de la balle en horizontale pour remettre la vitesse horizontale initiale
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)              

    ball.wrap_position(app.width, app.height)                                       # on s'assure de ne pas dessiner la balle hors des limites du jeu    


app.run()                                                                           # execution de l'application


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…